Analyse des données

Préparation des données avec pandas


In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import os
import sys
import pycountry

sys.path.append(os.path.abspath(".."))

pd.set_option("display.float_format", lambda x: f"{x :.2f}")
pd.set_option("display.max_columns", None)
from get_dataframes import GetDataframes

from tools import import_config

from plotting import (
    movies_by_decades,
    movies_by_genres,
    movies_by_duration_boxplot,
    movies_by_country,
    movies_duration_by_decades_boxplot,
    movies_top_x,
)

- On importe la dataframe "title_basics" qui contient les infos pour tout les films de la base de données.
- On enlève tout les films à caractère pornographique de notre liste.
- On nettoie les /N et les transforme en 0.
- On change les types de données des colonnes (float en int, etc...)
- On merge la dataframe "title_basics" avec "tmdb_full" pour obtenir des infos supplémentaires sur chaque film.
- On supprime les colonnes inutiles.
- On supprime les lignes pour lesquelles on a des valeurs manquantes suite au merging.
- On transforme les données dans les colonnes ayant plusieurs valeurs et les on les transforme en listes.
- On importe les dataframes "name_basics", "title_akas" et "title_principals".
- On merge la dataframe "title_akas" avec "tmdb_full".
- On filtre les films ayant eu une adaptaion en FR uniquement.
- On filtre seulement les films qui sont "Released".
- On nettoie "name_basics" comme précédement.
- On merge "name_basics", "title_principals" et on trie les acteurs, directeurs et autres dans un autre dataframe.
- On merge chaque dataframe (acteurs, directeurs, etc...) avec la liste des films qui étaient déjà merged avec tmdb.

In [2]:
import polars as pl

df = pd.read_parquet("../streamlit/datasets/site_web.parquet")
df.titre_genres.unique()

array(['Comédie', 'Action, Thriller, Crime', 'Romance, Comédie, Drame',
       ..., 'Drame, Histoire, Familial', 'Comédie, Action, Thriller',
       'Science-Fiction, Mystère, Drame, Thriller, Horreur'], dtype=object)

In [3]:
df[df.keywords.str.contains("christmas")]

budget                                       titre_genres  tmdb_id  \
164    30000000                                   Comédie, Romance   549053   
207           0                                   Romance, Comédie   295151   
213    27000000                            Drame, Comédie, Romance      245   
295    65000000                           Drame, Thriller, Mystère      345   
303    80000000                                Action, Fantastique      364   
...         ...                                                ...      ...   
4440  145000000  Familial, Animation, Fantastique, Action, Aven...    81188   
4553   45000000                                            Comédie   384682   
4579          0                                   Romance, Comédie   556803   
4588          0                                  Romance, Téléfilm   483104   
4610    1645000                                   Romance, Comédie   286554   

       titre_id                                           overview  \
164   tt8623904  Kate est une jeune femme abonnée aux mauvaises...   
207   tt1950235  Par une nuit de Noël glaciale, une petite vill...   
213   tt0276751  Will Freeman est un trentenaire londonien et u...   
295   tt0120663  Conviés à une soirée donnée par leur ami Victo...   
303   tt0103776  Abandonné à la naissance à cause de sa difform...   
...         ...                                                ...   
4440  tt1446192  Il existe dans le monde, dissimulés aux yeux d...   
4553  tt1711525  À la tête de l’entreprise familiale, Carole me...   
4579  tt8954732  En découvrant qu'elles se ressemblent comme de...   
4588  tt7608418  Pour cette jeune femme qui rêve de devenir jou...   
4610  tt2140619  Un couple se retrouve pris au piège lors d'une...   

      popularity production_countries  date    revenue  runtime  \
164        36.00             [GB, US]  2019  121550750      102   
207        24.79                 [US]  2019          0       93   
213        23.69     [FR, DE, GB, US]  2002  129000000      105   
295        48.11             [GB, US]  1999  162100000      159   
303        43.90                 [US]  1992  280000000      126   
...          ...                  ...   ...        ...      ...   
4440       82.84                 [US]  2012  306941670       97   
4553       34.77                 [US]  2016  114501299      105   
4579       34.62                 [US]  2018          0      101   
4588       19.15                 [US]  2017          0       92   
4610       20.51                 [US]  2014     868081       86   

                                                tagline  \
164                 Parfois tu dois juste avoir la foi.   
207                                                       
213                                                       
295                                                       
303             La Chauve-Souris, le Chat, le Pingouin.   
...                                                 ...   
4440                     Découvrez un monde de légende…   
4553  On avait dit juste une petite soirée de noël p...   
4579                                                      
4588                                                      
4610                      Avant l'amour, après le sexe.   

                    titre_str  rating_avg  rating_vote  \
164            Last Christmas        7.24         2172   
207           Flocons d'amour        6.09         1092   
213            Pour un garçon        6.66         2068   
295            Eyes Wide Shut        7.47         5647   
303          Batman : Le Défi        6.92         6051   
...                       ...         ...          ...   
4440        Les Cinq Légendes        7.36         6292   
4553          Joyeux bordel !        5.66         1924   
4579  La Princesse de Chicago        7.01         2109   
4588       A Christmas Prince        6.62         1378   
4610       Aventure d'un soir        6.43         1750   

        

In [4]:
def get_genre_df(df: pd.DataFrame, genres: str):
    sort_by = ["popularity", "rating_avg", "rating_vote"]
    ascending_ = [False for i in range(3)]
    condi = (
        (
            df["titre_genres"].str.contains(genres)
            & ~df["titre_genres"].str.contains("Animation")
        )
        if genres != "Animation"
        else df["titre_genres"].str.contains(genres)
    )
    return df[condi].sort_values(by=sort_by, ascending=ascending_)


drama_genre = get_genre_df(df, "Drame")
comedy_genre = get_genre_df(df, "Comédie")
horror_genre = get_genre_df(df, "Horreur")
animation_genre = get_genre_df(df, "Animation")
scifi_genre = get_genre_df(df, "Science-Fiction")
thriller_genre = get_genre_df(df, "Crime")
top = 10

In [5]:
config = import_config(add=True)
datas = GetDataframes(config)
link = "movies"
df = datas.get_dataframes(link)

2023-11-30 16:15:26 INFO     TSV files already exist.
2023-11-30 16:15:26 INFO     Parquet loaded ! Importing movies...
2023-11-30 16:15:26 INFO     Dataframe movies ready to use!


- On créée une visualisation pour voir la distribution des notes moyennes de chaque film, le nombre de films par décénnie et le total de votes par décénnie.

In [6]:
def iso_to_country_name(iso_codes):
    country_names = []
    for code in iso_codes:
        country = pycountry.countries.get(alpha_2=code)
        if country:
            country_names.append(country.name)
        else:
            country_names.append("Inconnu")
    return ", ".join(country_names)


df["production_countries"] = df["production_countries"].apply(
    iso_to_country_name
)

In [7]:
df[df["titre_duree"] == df["titre_duree"].max()]

titre_id titre_str  titre_date_sortie  titre_duree      titre_genres  \
45473  tt3854496  Ambiancé               2020        43200  [Fantasy, Drama]   

       rating_avg  rating_votes original_language original_title  popularity  \
45473        3.40           116                sv       Ambiancé        1.82   

      production_countries  revenue spoken_languages    status region   cuts  
45473               Sweden     0.00             [sv]  Released     FR  >2020

In [8]:
figs = movies_by_decades(df)

for fig in figs:
    fig.show()

- On cherche quel genre de films est le plus répandu dans le dataset filtré

In [9]:
movies_by_genres(df)

- On peut voir que le nombre de films en dessous des années 1960 est très bas et nous décidons de ne pas les garder.
- Les films ayant moins de 5 en note moyenne ne nous intéressent pas non plus.
- Et les films ayant reçu trop peu de votes sont aussi enlevés.

In [10]:
condi = (
    (df["titre_date_sortie"] >= 1960)
    & (df["rating_avg"] >= 6.3)
    & (df["rating_votes"] >= 5004)
)

df = df[condi].reset_index(drop="index")

- Ici nous filtrons les genres qui n'ont que peu d'interet pour le cinéma.

In [11]:
genres_ = ["Music", "Documentary", "Reality-TV", "News"]
df = df[
    df["titre_genres"].apply(lambda x: all(g not in x for g in genres_))
]

In [12]:
movies_by_genres(df)

- On peut désormais ré-afficher la distribution des notes moyennes, le total des films par décénnie et le nombre moyen des votes par décénnies

In [13]:
figs = movies_by_decades(df)

for fig in figs:
    fig.show()

- On cherche des informations sur la durée des films en minutes

In [14]:
movies_by_duration_boxplot(df)

- On constate qu'il y a beaucoup trop de films avec des durées bien trop longues ainsi que des films trop courts qui devraient être considéré plutot comme des court-métrages.
- On enlève donc les films en dessous de 62 minutes et au dessus de 210 minutes (3h30)

In [15]:
condi = ~((df["titre_duree"] < 62) | (df["titre_duree"] > 210))
df = df[condi].reset_index(drop="index")

movies_by_duration_boxplot(df)

- On cherche les 10 premiers pays desquels proviennent les films du dataset.

In [16]:
movies_by_country(df)

- On cherche si il y a une augmentation ou diminution de la durée des films dans le temps.

In [17]:
movies_duration_by_decades_boxplot(df)

In [18]:
# df[((df["titre_duree"] > 170) & (df["titre_date_sortie"] > 2020))]

- On cherche les 10 films avec le plus de votes sans compter les documentaires, concerts et films d'animations.

In [19]:
movies_top_x(df, top=10)

In [20]:
# #  World record of the longest documentary
# # source :https://fr.wikipedia.org/wiki/Ambianc%C3%A9
# longest = actor[actor["titre_duree"] == 43200]
# n = longest[["titre_str", "titre_duree"]].iloc[0]
# heure = n["titre_duree"] // 60
# jours = heure // 24
# print(f"Le film le plus long est {n['titre_str']} d'une durée de {heure} heures soit {jours} jours")
# # All time median for duration
# print(f"All time duration median : {round(actor['titre_duree'].median())} minutes")